## 實作案例: 請先切換kernel 於您新安裝的 kernel環境
## <span style="color:red">Change to kernel:   Image_XXXXXXXXXXl</span>.

https://python.langchain.com.cn/docs/get_started/quickstart

In [1]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

## 安裝專屬套件 (光碟內沒有的套件)

In [3]:
#!pip install xxx

## LOAD LIBRARY

In [4]:
# LOAD LIBRARY
from torch import cuda, bfloat16, float16
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig, pipeline, TextStreamer

## 下載模型

## 主程式  (請特別關注 llm)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

MODEL_ID = "/work/u00cjz00/slurm_jobs/github/models/Llama-2-7B-Chat-GPTQ"
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={"temperature": 0})

In [6]:
llm("Explain the difference between ChatGPT and open source LLMs in a couple of lines.")

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'\n\nChatGPT is a proprietary language model developed by Meta AI that has been fine-tuned on a massive dataset of text from the internet to generate human-like responses to a wide range of prompts. In contrast, open source LLMs are pre-trained language models that have been made publicly available for anyone to use, modify, and improve upon. These models are typically trained on large datasets of text from various sources, such as books, articles, and websites, and can be used for a variety of natural language processing tasks, including text classification, sentiment analysis, and machine translation.'

In [7]:
### Prompt Template
prompt_template = '''
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{question} [/INST]

'''

prompt = PromptTemplate(
    input_variables=["question"],
    template=prompt_template,
)

In [8]:
question = "Explain what are Deep Neural Networks in 2-3 sentences"
print(prompt.format(question=question))


<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Explain what are Deep Neural Networks in 2-3 sentences [/INST]




In [9]:
llm(prompt.format(question=question))

'Deep neural networks (DNNs) are a type of machine learning model that use artificial neural networks to analyze and process data. They consist of multiple layers of interconnected nodes or "neurons" that learn to recognize patterns in large datasets, allowing them to perform tasks such as image recognition, natural language processing, and speech recognition. DNNs are called "deep" because they have many layers, which enables them to learn complex representations of data.'

In [10]:
## Create Chain01
from langchain.chains import LLMChain
chain01 = LLMChain(llm=llm, prompt=prompt)
result01 = chain01.run(question=question)
print(result01.strip())

Deep neural networks (DNNs) are a type of artificial neural network (ANN) designed to recognize patterns in complex data sets, such as images, speech, and natural language. DNNs consist of multiple layers of interconnected nodes or "neurons" that process inputs and produce outputs, allowing them to learn and represent increasingly abstract representations of the input data. These networks can be trained using large amounts of data and have been shown to achieve state-of-the-art performance in various applications, including computer vision, natural language processing, and speech recognition.


In [11]:
## Create Chain02
### Prompt Template for summary
prompt_template = "<s>[INST] Use the summary {summary} and give 3 examples of practical applications with 1 sentence explaining each [/INST]"

prompt02 = PromptTemplate(
    input_variables=["summary"],
    template=prompt_template,
)
chain02 = LLMChain(llm=llm, prompt=prompt02)
print(prompt02.format(summary=result01))
result02 = chain02.run(result01)
print(result02.strip())

<s>[INST] Use the summary Deep neural networks (DNNs) are a type of artificial neural network (ANN) designed to recognize patterns in complex data sets, such as images, speech, and natural language. DNNs consist of multiple layers of interconnected nodes or "neurons" that process inputs and produce outputs, allowing them to learn and represent increasingly abstract representations of the input data. These networks can be trained using large amounts of data and have been shown to achieve state-of-the-art performance in various applications, including computer vision, natural language processing, and speech recognition. and give 3 examples of practical applications with 1 sentence explaining each [/INST]
Sure! Here is a summary of deep neural networks (DNNs), along with three examples of their practical applications:
Deep Neural Networks (DNNs):
A Deep Neural Network (DNN) is a type of artificial neural network (ANN) designed to recognize patterns in complex data sets, such as images, sp

In [12]:
## Chaining Chains
## multi_chain
multi_chain = SimpleSequentialChain(chains=[chain01, chain02], verbose=True)
result_mutiple = multi_chain.run(question)
print(result_mutiple.strip())



> Entering new SimpleSequentialChain chain...
Deep neural networks (DNNs) are a type of artificial neural network (ANN) that are designed to recognize complex patterns in data by learning from large datasets. These networks consist of multiple layers of interconnected nodes or "neurons," which process and transmit information between each other, allowing the network to learn and make predictions about input data. DNNs have been shown to be highly effective at tasks such as image recognition, natural language processing, and speech recognition, due to their ability to learn and adapt to large amounts of data.
  Sure! Here are three practical applications of deep neural networks:

1. Image Recognition: Deep neural networks can be used to recognize objects within images by learning patterns within large datasets of labeled images. For example, Google's DeepMind developed a DNN called AlphaGo that was able to defeat a world champion Go player by using a DNN to analyze positions on a Go b